<a href="https://colab.research.google.com/github/SergioManuelJob/ExpertSystemCLIPS/blob/master/CLIPS_Ruta_Mas_Corta.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install clipspy

# Importacion de clips
import clips
import logging

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 891.6/891.6 kB 6.7 MB/s eta 0:00:00


Prueba de ClipsPy

In [ ]:
# Preparación del entorno
logging.basicConfig(level=logging.DEBUG,
                    force=True,
                    format='%(message)s'
                    )

In [ ]:
DEFTEMPLATE_NODO = """
(deftemplate nodo
   (slot nombre (type SYMBOL))
   (slot vecino (type SYMBOL))
   (slot distancia (type INTEGER))
)"""

DEFTEMPLATE_NODOINICIAL = """
(deftemplate nodoInicial
   (slot nombre)
)
"""

DEFTEMPLATE_RUTA = """
(deftemplate ruta
   (slot inicial (type SYMBOL))
   (slot final (type SYMBOL))
   (slot costo (type INTEGER))
)
"""

In [ ]:
DEFFACTS_NODOS = """
(deffacts nodos-iniciales
   (nodo (nombre A) (vecino B) (distancia 10))
   (nodo (nombre A) (vecino C) (distancia 30))
   (nodo (nombre A) (vecino D) (distancia 100))
   (nodo (nombre B) (vecino C) (distancia 5))
   (nodo (nombre C) (vecino D) (distancia 7))
)
"""

DEFFACTS_NODOINICIAL = """
(deffacts nodo-inicial
   (nodoInicial (nombre A))
)
"""

In [ ]:
DEFRULE_ENCONTRARRUTA= """
(defrule encontrar-ruta
   ?nodoInicial <- (nodoInicial (nombre ?inicio))
   ?nodoVecino <- (nodo (nombre ?inicio) (vecino ?vecino) (distancia ?distancia))
   =>
   (printout t "Calculando la mejor ruta desde el nodo " ?inicio " hacia otros nodos:" crlf)
   (printout t "  " ?vecino " (Costo: " ?distancia ")" crlf)
   (assert (ruta (inicial ?inicio) (final ?vecino) (costo ?distancia)))
)
"""

DEFRULE_ENCONTRARRUTAINDIRECTA = """
(defrule encontrar-rutas-indirectas
   ?ruta <- (ruta (inicial ?inicio) (final ?intermedio) (costo ?costo1))
   ?nodoIntermedio <- (nodo (nombre ?intermedio) (vecino ?final) (distancia ?distancia2))
   (test (not (eq ?intermedio ?inicio)))
   =>
   (bind ?costoTotal (+ ?costo1 ?distancia2))
   (printout t "  " ?final " (Costo: " ?costoTotal ")" crlf)
   (assert (ruta (inicial ?inicio) (final ?final) (costo ?costoTotal)))
   (retract ?ruta)
)
"""

In [ ]:
env = clips.Environment()
router = clips.LoggingRouter()
env.add_router(router)

env.build(DEFTEMPLATE_NODO)
env.build(DEFTEMPLATE_NODOINICIAL)
env.build(DEFTEMPLATE_RUTA)
env.build(DEFFACTS_NODOS)
env.build(DEFFACTS_NODOINICIAL)
env.build(DEFRULE_ENCONTRARRUTA)
env.build(DEFRULE_ENCONTRARRUTAINDIRECTA)
env.reset()

template = env.find_template('nodoInicial')

print("Reglas:\n")
for rule in env.rules():
    print(rule)

print("\nFacts:\n")
for fact in env.facts():
    print(fact)

Reglas:

(defrule MAIN::encontrar-ruta ?nodoInicial <- (nodoInicial (nombre ?inicio)) ?nodoVecino <- (nodo (nombre ?inicio) (vecino ?vecino) (distancia ?distancia)) => (printout t "Calculando la mejor ruta desde el nodo " ?inicio " hacia otros nodos:" crlf) (printout t " " ?vecino " (Costo: " ?distancia ")" crlf) (assert (ruta (inicial ?inicio) (final ?vecino) (costo ?distancia))))
(defrule MAIN::encontrar-rutas-indirectas ?ruta <- (ruta (inicial ?inicio) (final ?intermedio) (costo ?costo1)) ?nodoIntermedio <- (nodo (nombre ?intermedio) (vecino ?final) (distancia ?distancia2)) (test (not (eq ?intermedio ?inicio))) => (bind ?costoTotal (+ ?costo1 ?distancia2)) (printout t " " ?final " (Costo: " ?costoTotal ")" crlf) (assert (ruta (inicial ?inicio) (final ?final) (costo ?costoTotal))) (retract ?ruta))

Facts:

(nodo (nombre A) (vecino B) (distancia 10))
(nodo (nombre A) (vecino C) (distancia 30))
(nodo (nombre A) (vecino D) (distancia 100))
(nodo (nombre B) (vecino C) (distancia 5))
(nod

In [ ]:
env.run()

Calculando la mejor ruta desde el nodo A hacia otros nodos:
  D (Costo: 100)
Calculando la mejor ruta desde el nodo A hacia otros nodos:
  C (Costo: 30)
  D (Costo: 37)
Calculando la mejor ruta desde el nodo A hacia otros nodos:
  B (Costo: 10)
  C (Costo: 15)
  D (Costo: 22)


6